# Sentiment Analysis with RNN

In [65]:
# Import and data read

import numpy as np
import tensorflow as tf

with open('reviews.txt', 'r') as f:
  reviews = f.read()
with open('labels.txt', 'r') as f:
  labels = f.read()

In [66]:
reviews[:200]

'bromwell high is a cartoon comedy . it ran at the same time as some other programs about school life  such as  teachers  . my   years in the teaching profession lead me to believe that bromwell high  '

In [67]:
# Removing punctuation

from string import punctuation

all_text = ''.join([c for c in reviews if c not in punctuation])
reviews = all_text.split('\n')

all_text = ' '.join(reviews)
words = all_text.split()

all_text[:200]

'bromwell high is a cartoon comedy  it ran at the same time as some other programs about school life  such as  teachers   my   years in the teaching profession lead me to believe that bromwell high  s '

In [68]:
# Word examples

words[:5]

['bromwell', 'high', 'is', 'a', 'cartoon']

In [70]:
# Word2int dictionary
vocab_to_int = {k:v for v, k in enumerate(set(words), start = 1)}

# Reviews to integers
reviews_ints = [[vocab_to_int[word] for word in review.split(' ') if word in vocab_to_int.keys()] for review in reviews]

# Labels to integers
labels = [1 if label == 'positive' else 0 for label in labels.split('\n')]

In [82]:
from collections import Counter

review_lens = Counter([len(x) for x in reviews_ints])
print('Zero-length reviews: {}'.format(review_lens[0]))
print('Maximum review length: {}'.format(max(review_lens)))

Zero-length reviews: 0
Maximum review length: 2514


In [85]:
# Remove a review with the length of 0
reviews_ints = [i for i in reviews_ints if len(i) > 0]

review_lens = Counter([len(x) for x in reviews_ints])
print('Zero-length reviews: {}'.format(review_lens[0]))
print('Maximum review length: {}'.format(max(review_lens)))

Zero-length reviews: 0
Maximum review length: 2514


In [87]:
# Sequence padding (pre)

seq_len = 200

features = []
for review in reviews_ints:
  if len(review) > 200:
    features.append(review[:seq_len])
  else:
    while len(review) < 200:
      review.insert(0, 0)
    features.append(review)

In [95]:
len(features[2])

200

In [ ]:
!git commit -am "maximum and minimum review lengths"